In [1]:
import sys
sys.path.append('/mydata/universal-robust-attack/')
import os.path as osp
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision
import torchvision.datasets as tvdatasets
from torchvision.transforms import transforms

import modelzoo.zoo as zoo
import datasets
import config as cfg

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:1" if use_cuda else "cpu")
if use_cuda:
    print(torch.cuda.get_device_name(0))

GeForce GTX 1080 Ti


In [2]:
dataset_name = "ImageNet1k"
modelfamily = datasets.dataset_to_modelfamily[dataset_name]
dataset = datasets.__dict__[dataset_name]

In [3]:
idx2name = {403:"aircraft_carrier",
            407:"ambulance",
            466:"bullet_train",
            764:"rifle",
            779:"school_bus"}

candidates = [403, 407, 466, 764, 779]

In [4]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
trainset = dataset(train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=1, num_workers=10)
testset = dataset(train=False, transform=transform)
testloader = DataLoader(testset, batch_size=1, num_workers=10)

=> done loading ImageNet1k (train) with 1024934 examples
=> done loading ImageNet1k (test) with 256233 examples


In [5]:
collections = {target:{"data":[], "targets":[]} for target in candidates}
for image, label in testloader:
    if label.item() in candidates:
        collections[label.item()]["data"].append(image.clone())
        collections[label.item()]["targets"].append(label.clone())

In [6]:
for target, class_set in collections.items():
    class_name = idx2name[target]
    data, targets = torch.cat(class_set["data"]), torch.cat(class_set["targets"]) 
    torch.save((data, targets), f"./data/{class_name}_val.pt")
    print(f"Save to ./data/{class_name}_val.pt ({data.size(0)} samples).")

Save to ./data/aircraft_carrier_val.pt (256 samples).
Save to ./data/ambulance_val.pt (273 samples).
Save to ./data/bullet_train_val.pt (281 samples).
Save to ./data/rifle_val.pt (239 samples).
Save to ./data/school_bus_val.pt (248 samples).


In [7]:
collections = {target:{"data":[], "targets":[]} for target in candidates}
for image, label in trainloader:
    if label.item() in candidates:
        collections[label.item()]["data"].append(image.clone())
        collections[label.item()]["targets"].append(label.clone())

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [8]:
for target, class_set in collections.items():
    class_name = idx2name[target]
    data, targets = torch.cat(class_set["data"]), torch.cat(class_set["targets"]) 
    torch.save((data, targets), f"./data/{class_name}_train.pt")
    print(f"Save to ./data/{class_name}_train.pt ({data.size(0)} samples).")

Save to ./data/aircraft_carrier_train.pt (1044 samples).
Save to ./data/ambulance_train.pt (1027 samples).
Save to ./data/bullet_train_train.pt (1019 samples).
Save to ./data/rifle_train.pt (1061 samples).
Save to ./data/school_bus_train.pt (1052 samples).
